In [101]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LeakyReLU, Input, concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l1_l2


import glob
import os

from scipy.stats import zscore

In [88]:
weather_path = os.path.join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
clients_path = os.path.join('..','data','wifi','**','Clients per day.csv')
clients_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                           glob.glob(clients_path)), ignore_index=True)
weekday_labels = []
for d in range(7):
    label = 'weekday-%i' % d
    weekday_labels.append(label)
    clients_df[label] = [int(dt.weekday()==d) for dt in clients_df['Time']]
# Put clients date in the same format as it is in the weather data
clients_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in clients_df['Time']]

# This merge ignores the missing data point in the weather data
all_data = clients_df.merge(weather_df, left_on='Time', right_on='DATE')

cols_to_norm = ['TMIN', 'TMAX', 'PRCP']
for col in cols_to_norm:
    all_data[col] = zscore(all_data[col])

x_labels = weekday_labels + ['PRCP', 'TMAX', 'TMIN']
day_x=all_data[weekday_labels].values
weather_x=all_data[['PRCP', 'TMAX', 'TMIN']].values
date_x=np.expand_dims(np.arange(0,353/365,1/365).astype('float32'),axis=1)
y_labels = ['# Clients']
x = all_data[x_labels].values
y = all_data[y_labels].values
all_data

,Time,# Clients,weekday-0,weekday-1,weekday-2,weekday-3,weekday-4,weekday-5,weekday-6,DATE,PRCP,TMAX,TMIN
0,2017-01-01,93,0,0,0,0,0,0,1,2017-01-01,-0.343154,-0.820006,-0.792066
1,2017-01-02,73,1,0,0,0,0,0,0,2017-01-02,0.579624,-1.667506,-0.792066
2,2017-01-03,87,0,1,0,0,0,0,0,2017-01-03,1.589692,-1.067194,-0.792066
3,2017-01-04,97,0,0,1,0,0,0,0,2017-01-04,-0.343154,-0.725840,-1.262831
4,2017-01-05,92,0,0,0,1,0,0,0,2017-01-05,-0.343154,-2.138339,-1.427599
5,2017-01-06,88,0,0,0,0,1,0,0,2017-01-06,-0.218454,-2.208964,-1.757135
6,2017-01-07,39,0,0,0,0,0,1,0,2017-01-07,0.592094,-2.832818,-2.580974
7,2017-01-08,40,0,0,0,0,0,0,1,2017-01-08,-0.318214,-2.997609,-2.828126
8,2017-01-09,74,1,0,0,0,0,0,0,2017-01-09,-0.343154,-2.950526,-2.710435
9,2017-01-10,67,0,1,0,0,0,0,0,2017-01-10,-0.343154,-2.020631,-2.322053


In [115]:
epochs = 1000
reg=None#l1_l2(0.01,0.02)

def stackLayers(layerSet):
    #print (layerSet)
    stack = layerSet[0]
    if len(layerSet)>1:
        for i in range (0,len(layerSet)-1):
            stack=layerSet[i+1](stack)
    return (stack)
weatherLayers=[]
weatherLayers+=[Input((3,))]
weatherLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(4, activation='relu',kernel_regularizer=reg)]
weatherOut=stackLayers(weatherLayers)
print (weatherOut)

dayLayers=[]
dayLayers+=[Input((7,))]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
dayOut=stackLayers(dayLayers)

dateLayers=[]
dateLayers+=[Input((1,))]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
dateOut=stackLayers(dateLayers)

mainLayers=[]
mainLayers+=[concatenate([weatherOut,dayOut])]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(1, activation='linear')]
mainOut=stackLayers(mainLayers)

model=Model(inputs=[weatherLayers[0],dayLayers[0],dateLayers[0]],outputs=[mainOut])
model.compile(loss='mean_absolute_percentage_error',
              optimizer=Adam(0.0001))

model.summary()

early_stopping = EarlyStopping(patience=50, verbose=1)
history = model.fit([weather_x,day_x,date_x], [y],
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
from math import ceil



Tensor("dense_607/Relu:0", shape=(?, 4), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_88 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
input_89 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
dense_604 (Dense)               (None, 10)           40          input_88[0][0]                   
__________________________________________________________________________________________________
dense_608 (Dense)               (None, 10)           80          input_89[0][0]                   
_____________________________________________________

317/317 [==============================] - 0s 89us/step - loss: 99.5441 - val_loss: 99.4609
Epoch 28/1000
317/317 [==============================] - 0s 95us/step - loss: 99.4541 - val_loss: 99.4383
Epoch 29/1000
317/317 [==============================] - 0s 85us/step - loss: 99.6554 - val_loss: 99.4166
Epoch 30/1000
317/317 [==============================] - 0s 85us/step - loss: 99.6920 - val_loss: 99.3959
Epoch 31/1000
317/317 [==============================] - 0s 87us/step - loss: 99.3795 - val_loss: 99.3719
Epoch 32/1000
317/317 [==============================] - 0s 89us/step - loss: 99.4461 - val_loss: 99.3451
Epoch 33/1000
317/317 [==============================] - 0s 82us/step - loss: 99.5385 - val_loss: 99.3168
Epoch 34/1000
317/317 [==============================] - 0s 85us/step - loss: 99.4209 - val_loss: 99.2907
Epoch 35/1000
317/317 [==============================] - 0s 96us/step - loss: 99.5586 - val_loss: 99.2634
Epoch 36/1000
317/317 [==============================] - 0s 

317/317 [==============================] - 0s 95us/step - loss: 92.3607 - val_loss: 88.7387
Epoch 105/1000
317/317 [==============================] - 0s 92us/step - loss: 93.0479 - val_loss: 88.5069
Epoch 106/1000
317/317 [==============================] - 0s 95us/step - loss: 91.7254 - val_loss: 88.2583
Epoch 107/1000
317/317 [==============================] - 0s 98us/step - loss: 92.2747 - val_loss: 88.0043
Epoch 108/1000
317/317 [==============================] - 0s 98us/step - loss: 91.1652 - val_loss: 87.7725
Epoch 109/1000
317/317 [==============================] - 0s 89us/step - loss: 91.6830 - val_loss: 87.5213
Epoch 110/1000
317/317 [==============================] - 0s 95us/step - loss: 90.4156 - val_loss: 87.2614
Epoch 111/1000
317/317 [==============================] - 0s 92us/step - loss: 90.7448 - val_loss: 86.9905
Epoch 112/1000
317/317 [==============================] - 0s 92us/step - loss: 90.9125 - val_loss: 86.7318
Epoch 113/1000
317/317 [============================

317/317 [==============================] - 0s 98us/step - loss: 78.1716 - val_loss: 66.0360
Epoch 181/1000
317/317 [==============================] - 0s 95us/step - loss: 76.2657 - val_loss: 65.7213
Epoch 182/1000
317/317 [==============================] - 0s 92us/step - loss: 74.1403 - val_loss: 65.3004
Epoch 183/1000
317/317 [==============================] - 0s 87us/step - loss: 76.5658 - val_loss: 65.0570
Epoch 184/1000
317/317 [==============================] - 0s 92us/step - loss: 75.8787 - val_loss: 65.0029
Epoch 185/1000
317/317 [==============================] - 0s 112us/step - loss: 75.5819 - val_loss: 64.8371
Epoch 186/1000
317/317 [==============================] - 0s 98us/step - loss: 77.8483 - val_loss: 64.6398
Epoch 187/1000
317/317 [==============================] - 0s 95us/step - loss: 79.1861 - val_loss: 64.4576
Epoch 188/1000
317/317 [==============================] - 0s 100us/step - loss: 74.4326 - val_loss: 64.3041
Epoch 189/1000
317/317 [==========================

317/317 [==============================] - 0s 95us/step - loss: 73.9203 - val_loss: 60.3652
Epoch 257/1000
317/317 [==============================] - 0s 98us/step - loss: 72.9327 - val_loss: 60.3349
Epoch 258/1000
317/317 [==============================] - 0s 92us/step - loss: 73.2291 - val_loss: 60.2608
Epoch 259/1000
317/317 [==============================] - 0s 104us/step - loss: 76.7350 - val_loss: 60.1840
Epoch 260/1000
317/317 [==============================] - 0s 92us/step - loss: 72.9684 - val_loss: 60.1144
Epoch 261/1000
317/317 [==============================] - 0s 95us/step - loss: 71.3704 - val_loss: 60.0252
Epoch 262/1000
317/317 [==============================] - 0s 89us/step - loss: 74.3192 - val_loss: 59.9991
Epoch 263/1000
317/317 [==============================] - 0s 92us/step - loss: 68.9895 - val_loss: 59.9611
Epoch 264/1000
317/317 [==============================] - 0s 89us/step - loss: 70.1540 - val_loss: 59.8955
Epoch 265/1000
317/317 [===========================

...

317/317 [==============================] - 0s 98us/step - loss: 65.4168 - val_loss: 54.5096
Epoch 485/1000
317/317 [==============================] - 0s 95us/step - loss: 63.4724 - val_loss: 54.5038
Epoch 486/1000
317/317 [==============================] - 0s 98us/step - loss: 64.9358 - val_loss: 54.4949
Epoch 487/1000
317/317 [==============================] - 0s 98us/step - loss: 70.3149 - val_loss: 54.4795
Epoch 488/1000
317/317 [==============================] - 0s 92us/step - loss: 66.7263 - val_loss: 54.4685
Epoch 489/1000
317/317 [==============================] - 0s 89us/step - loss: 67.7062 - val_loss: 54.4199
Epoch 490/1000
317/317 [==============================] - 0s 98us/step - loss: 67.6343 - val_loss: 54.4001
Epoch 491/1000
317/317 [==============================] - 0s 96us/step - loss: 63.8862 - val_loss: 54.3535
Epoch 492/1000
317/317 [==============================] - 0s 92us/step - loss: 64.9895 - val_loss: 54.3131
Epoch 493/1000
317/317 [============================

317/317 [==============================] - 0s 101us/step - loss: 66.6392 - val_loss: 53.8196
Epoch 561/1000
317/317 [==============================] - 0s 95us/step - loss: 63.4086 - val_loss: 53.8096
Epoch 562/1000
317/317 [==============================] - 0s 95us/step - loss: 66.1259 - val_loss: 53.8071
Epoch 563/1000
317/317 [==============================] - 0s 92us/step - loss: 66.8573 - val_loss: 53.8177
Epoch 564/1000
317/317 [==============================] - 0s 111us/step - loss: 66.7674 - val_loss: 53.8218
Epoch 565/1000
317/317 [==============================] - 0s 95us/step - loss: 65.0727 - val_loss: 53.8054
Epoch 566/1000
317/317 [==============================] - 0s 95us/step - loss: 64.1899 - val_loss: 53.7994
Epoch 567/1000
317/317 [==============================] - 0s 98us/step - loss: 67.5375 - val_loss: 53.7934
Epoch 568/1000
317/317 [==============================] - 0s 101us/step - loss: 63.9443 - val_loss: 53.8026
Epoch 569/1000
317/317 [=========================

317/317 [==============================] - 0s 101us/step - loss: 61.6654 - val_loss: 53.4364
Epoch 637/1000
317/317 [==============================] - 0s 101us/step - loss: 64.7603 - val_loss: 53.4476
Epoch 638/1000
317/317 [==============================] - 0s 95us/step - loss: 64.8156 - val_loss: 53.4430
Epoch 639/1000
317/317 [==============================] - 0s 92us/step - loss: 64.6441 - val_loss: 53.4394
Epoch 640/1000
317/317 [==============================] - 0s 95us/step - loss: 61.6089 - val_loss: 53.4367
Epoch 641/1000
317/317 [==============================] - 0s 92us/step - loss: 64.1690 - val_loss: 53.4296
Epoch 642/1000
317/317 [==============================] - 0s 98us/step - loss: 62.7606 - val_loss: 53.4170
Epoch 643/1000
317/317 [==============================] - 0s 95us/step - loss: 63.6055 - val_loss: 53.4187
Epoch 644/1000
317/317 [==============================] - 0s 89us/step - loss: 65.2210 - val_loss: 53.4178
Epoch 645/1000
317/317 [==========================

317/317 [==============================] - 0s 95us/step - loss: 64.0847 - val_loss: 53.3400
Epoch 713/1000
317/317 [==============================] - 0s 104us/step - loss: 60.5413 - val_loss: 53.3305
Epoch 714/1000
317/317 [==============================] - 0s 95us/step - loss: 60.1924 - val_loss: 53.3285
Epoch 715/1000
317/317 [==============================] - 0s 95us/step - loss: 59.4137 - val_loss: 53.3193
Epoch 716/1000
317/317 [==============================] - 0s 95us/step - loss: 61.0394 - val_loss: 53.3201
Epoch 717/1000
317/317 [==============================] - 0s 95us/step - loss: 62.7392 - val_loss: 53.3087
Epoch 718/1000
317/317 [==============================] - 0s 98us/step - loss: 63.3582 - val_loss: 53.2942
Epoch 719/1000
317/317 [==============================] - 0s 95us/step - loss: 64.4828 - val_loss: 53.2862
Epoch 720/1000
317/317 [==============================] - 0s 108us/step - loss: 63.8720 - val_loss: 53.2869
Epoch 721/1000
317/317 [==========================

317/317 [==============================] - 0s 95us/step - loss: 58.7059 - val_loss: 53.4050
Epoch 789/1000
317/317 [==============================] - 0s 104us/step - loss: 63.5708 - val_loss: 53.4662
Epoch 790/1000
317/317 [==============================] - 0s 98us/step - loss: 65.6700 - val_loss: 53.4926
Epoch 791/1000
317/317 [==============================] - 0s 95us/step - loss: 62.2308 - val_loss: 53.4720
Epoch 792/1000
317/317 [==============================] - 0s 96us/step - loss: 63.3723 - val_loss: 53.4606
Epoch 793/1000
317/317 [==============================] - 0s 92us/step - loss: 65.1762 - val_loss: 53.4815
Epoch 794/1000
317/317 [==============================] - 0s 98us/step - loss: 62.2483 - val_loss: 53.4848
Epoch 795/1000
317/317 [==============================] - 0s 96us/step - loss: 63.8401 - val_loss: 53.4754
Epoch 796/1000
317/317 [==============================] - 0s 92us/step - loss: 58.9810 - val_loss: 53.4569
Epoch 797/1000
317/317 [===========================

In [107]:

evalSeg=317
print (weather_x[evalSeg:].shape)
model.evaluate(x=[weather_x[evalSeg:],day_x[evalSeg:],date_x[evalSeg:]],y=[y[evalSeg:]])

(36, 3)
36/36 [==============================] - 0s 362us/step


57.090850830078125

In [98]:
y

array([[ 93],
       [ 73],
       [ 87],
       [ 97],
       [ 92],
       [ 88],
       [ 39],
       [ 40],
       [ 74],
       [ 67],
       [ 88],
       [105],
       [150],
       [ 94],
       [ 74],
       [ 95],
       [ 83],
       [108],
       [104],
       [109],
       [122],
       [ 69],
       [ 55],
       [ 96],
       [135],
       [119],
       [ 97],
       [ 88],
       [ 61],
       [ 61],
       [138],
       [ 85],
       [ 92],
       [104],
       [ 87],
       [ 58],
       [ 98],
       [ 99],
       [154],
       [ 78],
       [115],
       [119],
       [132],
       [ 98],
       [ 89],
       [ 92],
       [ 81],
       [140],
       [162],
       [147],
       [138],
       [112],
       [ 94],
       [155],
       [191],
       [116],
       [ 75],
       [ 87],
       [ 93],
       [106],
       [110],
       [ 96],
       [ 97],
       [ 53],
       [ 82],
       [ 97],
       [127],
       [128],
       [101],
       [114],
       [ 62],
      